<a href="https://colab.research.google.com/github/jwengr/dacon/blob/main/%EC%86%8C%EC%84%A4%20%EC%9E%91%EA%B0%80%20%EB%B6%84%EB%A5%98%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EDA에서 특수문자의 사용빈도나 프랑스문자의 사용빈도가 결과 예측에
도움이 된 다는 것을 알 수 있었습니다.
일반적인 딥러닝에서의 텍스트분석은 주로 특수문자들을 없애거나 최소화하여
문장의 뜻을 맞춥니다.
하지만 우리는 주어진 텍스트에서 작가를 분류해야하며
주어진 텍스트는 서로 비슷합니다.
즉 텍스트간의 미묘한 차이를 구분하는 모델을 만들어야 하며, 특수문자나 프랑스문자에도 집중해야 할 것 입니다.

In [ ]:
!pip uninstall lightgbm

In [2]:
!git clone --recursive https://github.com/Microsoft/LightGBM

fatal: destination path 'LightGBM' already exists and is not an empty directory.


In [ ]:
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

In [ ]:
!pip install catboost

In [5]:
import pandas as pd
import numpy as np
import re
import xgboost as xgb
import catboost as ctb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import nltk

In [6]:
defaultpath = 'drive/My Drive/dacon/sosul/dataset'

In [7]:
train_df = pd.read_csv(defaultpath+'/train.csv',encoding='utf-8')

기본 전처리

In [8]:
train_df = train_df[train_df['text'].str.contains('\* \*')==False]

In [9]:
train_df['sentencelen'] = train_df['text'].apply(lambda x: len(x.split('.')))

In [10]:
train_df['charlen'] = train_df['text'].apply(lambda x: len(x))

In [11]:
train_df['c/s'] = train_df['charlen']/(train_df['sentencelen']+1)  ## 0으로 나뉘는것을 방지

In [12]:
train_df['upperlen'] = train_df['text'].apply(lambda x: len(re.findall('[A-Z]',x)))

In [13]:
train_df['u/s'] = train_df['upperlen']/(train_df['sentencelen']+1)  ## 0으로 나뉘는것을 방지

In [14]:
train_df['u/s'] = train_df['upperlen']/(train_df['charlen']+1)  ## 0으로 나뉘는것을 방지

프랑스어가 포함된 문장만 따로 추출

In [15]:
train_df_fr = train_df[train_df['text'].str.contains('[à|ä|ö|î|ù|â|Œ|ç|ê|ü|ñ|ô|Æ|œ|ë|æ|é|Ê|è|ì]')].copy()

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

char TF-IDF : 특수문자까지 포함하여

In [69]:
char_tfidfv  = TfidfVectorizer(analyzer='char').fit(train_df['text'])
enc = char_tfidfv.transform(train_df['text']).toarray()
char_tfidf_train_df = pd.concat([train_df.reset_index(),pd.DataFrame(enc)],axis=1).drop(['level_0','index','text','sentencelen','charlen','upperlen'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(char_tfidf_train_df.drop('author',axis=1), char_tfidf_train_df['author'],
                                                    test_size=0.2, random_state=2021,
                                                    stratify=char_tfidf_train_df['author']) 
xgb_model = xgb.XGBClassifier(num_class=5,objective='multi:softmax',tree_method='gpu_hist', gpu_id=0,n_estimators=10000)
xgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['merror','mlogloss'],early_stopping_rounds=1000,verbose=False)
lgb_model = lgb.LGBMClassifier(num_class=5,objective='multiclass',device_type='gpu',n_estimators=10000,early_stopping_round=1000)
lgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['multi_error','multi_logloss'],verbose=False)
ctb_model = ctb.CatBoostClassifier(n_estimators=10000,early_stopping_rounds=1000,task_type="GPU",loss_function='MultiClass')
ctb_model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=False)
print(f'xgb : {xgb_model.best_iteration,xgb_model.score(x_test,y_test)}')
print(f'lgb : {lgb_model.score(x_test,y_test)}')
print(f'ctb : {ctb_model.get_best_iteration(),ctb_model.score(x_test,y_test)}')

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


xgb : (1908, 0.5873985593143065)
lgb : 0.5929607002826662
ctb : (3694, 0.6029907905534786)


char tfidf : 프랑스어포함문장만

In [70]:
char_tfidfv  = TfidfVectorizer(analyzer='char').fit(train_df_fr['text'])
enc = char_tfidfv.transform(train_df_fr['text']).toarray()
char_tfidf_train_df = pd.concat([train_df_fr.reset_index(),pd.DataFrame(enc)],axis=1).drop(['level_0','index','text','sentencelen','charlen','upperlen'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(char_tfidf_train_df.drop('author',axis=1), char_tfidf_train_df['author'],
                                                    test_size=0.2, random_state=2021,
                                                    stratify=char_tfidf_train_df['author']) 
xgb_model = xgb.XGBClassifier(num_class=5,objective='multi:softmax',tree_method='gpu_hist', gpu_id=0,n_estimators=10000)
xgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['merror','mlogloss'],early_stopping_rounds=1000,verbose=False)
lgb_model = lgb.LGBMClassifier(num_class=5,objective='multiclass',device_type='gpu',n_estimators=10000,early_stopping_round=1000)
lgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['multi_error','multi_logloss'],verbose=False)
ctb_model = ctb.CatBoostClassifier(n_estimators=10000,early_stopping_rounds=1000,task_type="GPU",loss_function='MultiClass')
ctb_model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=False)
print(f'xgb : {xgb_model.best_iteration,xgb_model.score(x_test,y_test)}')
print(f'lgb : {lgb_model.score(x_test,y_test)}')
print(f'ctb : {ctb_model.get_best_iteration(),ctb_model.score(x_test,y_test)}')

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


xgb : (83, 0.8507462686567164)
lgb : 0.8656716417910447
ctb : (1064, 0.8059701492537313)


TF-IDF : 특수문자까지 포함하여 features 500

In [23]:
tfidfv = TfidfVectorizer(token_pattern="[a-zA-Z]+|[0-9]+|\W+",max_features= 500).fit(train_df['text'])
enc = tfidfv.transform(train_df['text']).toarray()
tfidf_train_df = pd.concat([train_df.reset_index(),pd.DataFrame(enc)],axis=1).drop(['level_0','index','text','sentencelen','charlen','upperlen'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(tfidf_train_df.drop('author',axis=1), tfidf_train_df['author'],
                                                    test_size=0.2, random_state=2021,
                                                    stratify=tfidf_train_df['author']) 
xgb_model = xgb.XGBClassifier(num_class=5,objective='multi:softmax',tree_method='gpu_hist', gpu_id=0,n_estimators=10000)
xgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['merror','mlogloss'],early_stopping_rounds=1000,verbose=False)
lgb_model = lgb.LGBMClassifier(num_class=5,objective='multiclass',device_type='gpu',n_estimators=10000,early_stopping_round=1000)
lgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['multi_error','multi_logloss'],verbose=False)
ctb_model = ctb.CatBoostClassifier(n_estimators=10000,early_stopping_rounds=1000,task_type="GPU",loss_function='MultiClass')
ctb_model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=False)
print(f'xgb : {xgb_model.best_iteration,xgb_model.score(x_test,y_test)}')
print(f'lgb : {lgb_model.score(x_test,y_test)}')
print(f'ctb : {ctb_model.get_best_iteration(),ctb_model.score(x_test,y_test)}')

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


xgb : (3586, 0.7334731467128659)
lgb : 0.737029269627063
ctb : (9999, 0.7435944196225038)


TF-IDF : 특수문자까지 포함하여 features 1000

In [22]:
tfidfv = TfidfVectorizer(token_pattern="[a-zA-Z]+|[0-9]+|\W+",max_features= 1000).fit(train_df['text'])
enc = tfidfv.transform(train_df['text']).toarray()
tfidf_train_df = pd.concat([train_df.reset_index(),pd.DataFrame(enc)],axis=1).drop(['level_0','index','text','sentencelen','charlen','upperlen'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(tfidf_train_df.drop('author',axis=1), tfidf_train_df['author'],
                                                    test_size=0.2, random_state=2021,
                                                    stratify=tfidf_train_df['author']) 
xgb_model = xgb.XGBClassifier(num_class=5,objective='multi:softmax',tree_method='gpu_hist', gpu_id=0,n_estimators=10000)
xgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['merror','mlogloss'],early_stopping_rounds=1000,verbose=False)
lgb_model = lgb.LGBMClassifier(num_class=5,objective='multiclass',device_type='gpu',n_estimators=10000,early_stopping_round=1000)
lgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['multi_error','multi_logloss'],verbose=False)
ctb_model = ctb.CatBoostClassifier(n_estimators=10000,early_stopping_rounds=1000,task_type="GPU",loss_function='MultiClass')
ctb_model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=False)
print(f'xgb : {xgb_model.best_iteration,xgb_model.score(x_test,y_test)}')
print(f'lgb : {lgb_model.score(x_test,y_test)}')
print(f'ctb : {ctb_model.get_best_iteration(),ctb_model.score(x_test,y_test)}')

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


xgb : (4728, 0.7597337466946293)
lgb : 0.7653870703018145
ctb : (9999, 0.7658429834959424)


tfidf feature 2000
float 64를 낮춰줍니다

In [47]:
tfidfv = TfidfVectorizer(token_pattern="[a-zA-Z]+|[0-9]+|\W+",max_features= 2000,dtype=np.float32).fit(train_df['text'])
enc = tfidfv.transform(train_df['text']).toarray().astype(np.float16)
tfidf_train_df = pd.concat([train_df.reset_index(),pd.DataFrame(enc)],axis=1).drop(['level_0','index','text','sentencelen','charlen','upperlen'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(tfidf_train_df.drop('author',axis=1), tfidf_train_df['author'],
                                                    test_size=0.2, random_state=2021,
                                                    stratify=tfidf_train_df['author']) 
xgb_model = xgb.XGBClassifier(num_class=5,objective='multi:softmax',tree_method='gpu_hist', gpu_id=0,n_estimators=20000)
xgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['merror','mlogloss'],early_stopping_rounds=1000,verbose=False)
lgb_model = lgb.LGBMClassifier(num_class=5,objective='multiclass',device_type='gpu',n_estimators=20000,early_stopping_round=1000)
lgb_model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=['multi_error','multi_logloss'],verbose=False)
ctb_model = ctb.CatBoostClassifier(n_estimators=20000,early_stopping_rounds=1000,task_type="GPU",loss_function='MultiClass')
ctb_model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=False)
print(f'xgb : {xgb_model.best_iteration,xgb_model.score(x_test,y_test)}')
print(f'lgb : {lgb_model.score(x_test,y_test)}')
print(f'ctb : {ctb_model.get_best_iteration(),ctb_model.score(x_test,y_test)}')

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


xgb : (6360, 0.7784261876538707)
lgb : 0.784261876538707
ctb : (19992, 0.7809793015409866)
